In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from python.fluorophores import FlStatic, FlCollection, FlCollectionBlinking
from python.psfs import PsfVectorial
from python.estimators import est_quad2Diter
from python.simulators import Simulator
from python.simulators import SimSequencefile
from python.tools import makeNPC

## Simulate several fluorophores in a fluorophore collection

In [ ]:
## two fluorophores
fl1 = FlStatic()
fl2 = FlStatic()

psf_vec = PsfVectorial()
psf_vec.setpinhole(AU=1)

fc = FlCollection()
fc.add([fl1, fl2])

sim=Simulator(fluorophores=fc) # make a simulator and attach fluorophore

In [ ]:
numberOfLocalizations = 1000
orbitpoints = 4
probecenter = True
L = 75
sim.definePattern("donut", psf_vec, 
                  phasemask = "vortex", 
                  makepattern = "orbitscan", 
                  orbitpoints = orbitpoints,
                  probecenter = probecenter,
                  orbitL = L,
                  laserpower = 100)
sim.defineComponent("estdonut", "estimator",est_quad2Diter, parameters=[L,probecenter], dim=(0,1))

In [ ]:
seq = ["donut", "estdonut"]
out = sim.runSequence(seq,maxlocs=numberOfLocalizations)
stats = sim.summarize_results(out)

In [ ]:
x = np.arange(0,750+20,20) # 0:20:750;

fig = plt.figure()
ax = fig.add_subplot(121)

sim.scan_fov(seq, x, dimplot=0, dimscan=0, fluorophorenumber=1,
             ax1=["std","rmse","sCRB","bias"],
             title="scan second fluorophore in x", 
             tag="x")

In [ ]:
# now z-dependence
fl1.pos = [0, 0, 0]
fl2.pos = [100, 0, 0]
z = np.arange(0, 1050, 50)  # 0:50:1000;

ax = fig.add_subplot(122)
sim.scan_fov(seq, z, dimplot=0, dimscan=2, fluorophorenumber=1,
             ax1=["std","rmse","sCRB","bias"],
             title="scan second fluorophore in z",
             tag="z")

## Imaging of blinking fluorophores with Abberior sequence


In [ ]:
# make abberior simulator
sim = SimSequencefile()
sim.posgalvo = [0, 0, 0]
sim.posEOD = [0, 0, 0]
sim.time = 0
sim.background = 0
fname = os.path.join(SCRIPT_DIR, "examples", "Imaging_2D.json")
sim.loadsequence(fname) # only sequence file, then simple gauss and donut PSFs are used (fast)
sim.makescoutingpattern(np.array([[-100, -100], [400, 250]]))

In [ ]:
# make fake NPCs

# make a fluorophore collection with blinking fluorophores


photonbudget=[800, 5000]
reactivations = [0, 3]
titles=["PALM", "dSTORM"]

for k in range(len(photonbudget)):
    fc = FlCollectionBlinking()
    # set parameterst for caged fluorophore, PAFP or similar
    laserpower = 5
    switchpar = {}
    switchpar['brightness'] = 100*laserpower
    switchpar['toffsmlm'] = 10*1e3  # on-switching time in ms
    switchpar['photonbudget'] = photonbudget[k]
    switchpar['tonsmlm'] = 1e8  #  ms stays on, only bleached
    switchpar['activations'] = reactivations[k] # re activations
    switchpar['starton'] = 0 # fluorophores start in random on / off state, determined by tonsmlm, toffsmlm
    fc.setpar(**switchpar)

    # add fake NPCs
    fc.addstatic(makeNPC(pos=[0, 0, 0]))
    # add more NPCs at positions dpos
    fc.addstatic(makeNPC(pos=[250, 50, 0]))
    fc.addstatic(makeNPC(pos=[50, 150, 0]))

    sim.fluorophores = fc
    out = sim.scoutingSequence(maxrep=5000)

    #plot results

    vld = (out.loc.vld==1) & (out.loc.itr==max(out.loc.itr))
    vldcfr = vld & (out.loc.cfr<0.1)
    notvld = ~vld & ~vldcfr
    fig, ax = plt.subplots(1,1)
    ax.plot(sim.scoutingcoordinates[:,0],sim.scoutingcoordinates[:,1],'k+')
    ax.plot(out.loc.xnm[notvld],out.loc.ynm[notvld],'c.')
    ax.plot(out.loc.xnm[vld],out.loc.ynm[vld],'m.')
    ax.plot(out.loc.xnm[vldcfr],out.loc.ynm[vldcfr],'bx')
    posfl=out.fluorophores.pos[-1,...].squeeze()
    ax.plot(posfl[:,0],posfl[:,1],'ro')
    ax.set_aspect('equal')
    ax.legend(['scouting','not vld', 'last itr vld','last itr vld +cfr', 'fluorophore'])
    ax.set_title(titles[k])
